In [2]:
%%capture
%pip install python-dotenv neo4j_tools

In [ ]:

# Neo4j Connection Setup
from neo4j import GraphDatabase

from dotenv import load_dotenv
import os
load_dotenv('ws.env', override=True)

# Replace these with your Neo4j credentials and connection URI
# Neo4j
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE')

# Create a Neo4j driver instance
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Your Cypher query

print("Checking the connection:")

# Helper function to run and display Cypher query results
def run_query(query):
    with driver.session() as session:
        result = session.run(query)
        # Collect results as a list
        records = [record.data() for record in result]
        # Print the records
        for record in records:
            print(record)
        return records

# Run the query; check the connection
query = "MATCH p=()-[]-() limit 1 RETURN p"
results = run_query(query)



Checking the connection:
{'p': [{'countryCode': 'US', 'companyName': 'ACME Biologics', 'iso3Code': 'USA'}, 'SUPPLIES_RM', {'generation': 'g1', 'productSKU': 'df4be3de-0c6b-4d2c-ac71-db4b9fbdf395', 'package': 'all', 'form': 'Tablet', 'strength': '5mg', 'materialType': 'RM', 'description': 'Iolescidib Tablet 5mg', 'location': 'Philadelphia PA/US', 'globalBrand': 'Iolescidib', 'rmSequence': 1}]}


## Understanding Pharma Supply Chain Model

In [6]:
query = "call db.schema.visualization()"
results = run_query(query)

{'nodes': [{'name': '_Bloom_Perspective_', 'indexes': [], 'constraints': ["Constraint( id=8, name='constraint_f7832722', type='NODE PROPERTY UNIQUENESS', schema=(:_Bloom_Perspective_ {id}), ownedIndex=7 )"]}, {'name': 'Site', 'indexes': [], 'constraints': ["Constraint( id=18, name='Site_nkey', type='NODE KEY', schema=(:Site {location, market}), ownedIndex=17 )"]}, {'name': 'ProcessStep', 'indexes': ['globalBrand,processStage,processStep', 'stageNum,stepNum', 'materialType,processStage,processStep', 'globalBrand,materialType,location,stageNum,stepNum'], 'constraints': ["Constraint( id=20, name='ProcessStep_nkey', type='NODE KEY', schema=(:ProcessStep {globalBrand, location, materialType, package, processStage, processStep}), ownedIndex=36 )"]}, {'name': 'FG', 'indexes': [], 'constraints': []}, {'name': 'Recipe', 'indexes': [], 'constraints': ["Constraint( id=25, name='Recipe_nkey', type='NODE KEY', schema=(:Recipe {globalBrand, form, strength, generation, market}), ownedIndex=24 )"]}, {

## Dependency Chain: Suppliers->Products->Distributors

In [7]:
query = "MATCH path = (sup:Suppliers)-[:SUPPLIES_RM]->(rm:RM)-[:PRODUCT_FLOW*]->(prod:Product)-[:DISTRIBUTED_BY]->(dist:Distributor) WHERE prod.productSKU = '7e882292-ae98-45eb-8119-596b5d8b73e1' RETURN nodes(path) AS nodes, relationships(path) AS relationships;"
results = run_query(query)

{'nodes': [{'countryCode': 'CN', 'companyName': 'Yuhan Corporation', 'iso3Code': 'CHN'}, {'generation': 'g2', 'productSKU': '1d1b2379-2137-4c94-9107-1e2b16f6fae3', 'package': 'blister pack', 'form': 'Caplet', 'strength': '20mg', 'materialType': 'RM', 'description': 'Calciiarottecarin Caplet 20mg', 'globalBrand': 'Calciiarottecarin', 'location': 'Antwerp/BE', 'rmSequence': 2}, {'generation': 'g2', 'productSKU': '69d57a9d-6ec2-40ff-8128-52584965d62b', 'package': 'blister pack', 'form': 'Caplet', 'strength': '20mg', 'materialType': 'FG', 'description': 'Calciiarottecarin Caplet 20mg', 'globalBrand': 'Calciiarottecarin', 'location': 'Antwerp/BE'}, {'generation': 'g2', 'productSKU': '327bc085-2afa-40b0-935a-63fbba7fa693', 'package': 'all', 'form': 'Caplet', 'strength': '20mg', 'materialType': 'DIST', 'description': 'Calciiarottecarin Caplet 20mg', 'globalBrand': 'Calciiarottecarin', 'location': 'Paris/FR'}, {'generation': 'g2', 'sourceDate': neo4j.time.Date(2024, 11, 22), 'productSKU': '7e8

## Dependency Chain given a Supplier

In [ ]:
// Find dependency chain from Suppliers -> RM -> API -> Finished Goods-> Distributor for a given Supplier 
MATCH path = (sup:Suppliers)-[:SUPPLIES_RM]->(rm:RM)-[:PRODUCT_FLOW*]->(fg:DIST)-[:DISTRIBUTED_BY]->(dist:Distributor)
where sup.companyName = 'Palmetto Pharmaceuticals'

RETURN path;

## (Bottleneck) Find RMs with limited Supplier:

In [ ]:
// Find materials with limited supplier redundancy:
MATCH (rm:RM)<-[:SUPPLIES_RM]-(sup:Suppliers)
WITH rm, COUNT(sup) AS supplierCount
WHERE supplierCount = 1
RETURN rm.productSKU AS rawMaterialSKU, 
       rm.globalBrand AS rawMaterialName, 
       supplierCount
ORDER BY supplierCount

## (Bottleneck)APIs that impact multiple Drug Products:

In [ ]:
// APIs used in multiple Drug Products may create a scarcity bottleneck
// API = Active Pharmaceutical Ingredient 
MATCH (api:API)-[:PRODUCT_FLOW]->(dp:DP)
WITH api, COUNT(dp) AS productCount
WHERE productCount > 4  // Arbitrary threshold for major dependency
RETURN api.productSKU AS apiSKU, 
       api.globalBrand AS apiName, 
       productCount
ORDER BY productCount DESC

## High Dependecy Risk with just 1 supplier

In [ ]:
// 1. Find APIs used in multiple Drug Products (scarcity risk)
// 2. Check how many suppliers provide each API = Active Pharmaceutical Ingredient
// 3. Flag APIs that have only one supplier (high dependency risk)

MATCH (sup:Suppliers)-[:SUPPLIES_RM]->(rm:RM)-[:PRODUCT_FLOW]->(api:API)-[:PRODUCT_FLOW]->(dp:DP)
WITH api, COUNT(dp) AS productCount, COLLECT(DISTINCT dp) AS dpList, COUNT(DISTINCT sup) AS supplierCount, COLLECT(DISTINCT sup.companyName) AS supplierList
WHERE productCount > 4  // Arbitrary threshold for major dependency
RETURN api.productSKU AS apiSKU, 
       api.globalBrand AS apiName, 
       productCount, 
       supplierCount, 
       supplierList,
       CASE 
           WHEN supplierCount = 1 THEN '⚠️ Single Supplier Bottleneck!!** '
           ELSE 'Multiple Suppliers Available'
       END AS SupplierRisk
ORDER BY productCount DESC;

## ShotestPath from RM-> Product (DIST) with distributor

In [ ]:
// Find the shortest path from raw materials to a particular finished product SKU. Show the distributor at the end just to pinpoint the end of the path! 
MATCH path = shortestPath((s:RM)-[:PRODUCT_FLOW* ]->(p:Product {productSKU: "7e882292-ae98-45eb-8119-596b5d8b73e1"}))
OPTIONAL MATCH (d:Distributor)<-[dist:DISTRIBUTED_BY]-(p)
RETURN path,dist, d

## Raw Materials Demand Back Pressure

In [ ]:
// Determine the raw material quantity required to fulfill demand at the distributor level, considering the entire supply chain (from raw materials → production → distribution). It also identifies suppliers of these raw materials and how the demand flows backward through the supply network
// * Designed to analyze Raw Materials Demand Back Pressure, which essentially tracks how demand at the distribution level affects raw material needs upstream in the supply chain


MATCH (d:Distributor)<-[db:DISTRIBUTED_BY]-(prod:Product)
WHERE prod.productSKU = '9a6b431f-3a38-4b45-9451-fbf39b2e2fd0'

MATCH (api)<-[pf2:PRODUCT_FLOW]-(rm:RM)
WHERE pf2.globalBrand = prod.globalBrand
  AND pf2.strength = prod.strength
  AND pf2.form = prod.form
  AND pf2.generation = prod.generation

WITH d, db.demandQty AS demandQty, db, prod, COLLECT(DISTINCT rm) AS RMList
UNWIND RMList AS curRM

MATCH p = shortestPath((prod)<-[pf1:PRODUCT_FLOW*]-(curRM))
MATCH p3 = (myProd:Product)<-[pf:PRODUCT_FLOW]-(curRM)<-[:SUPPLIES_RM]-(sup:Suppliers)
WHERE pf.globalBrand = prod.globalBrand
  AND pf.strength = prod.strength
  AND pf.form = prod.form
  AND pf.generation = prod.generation
  AND pf.market = d.market

RETURN sup.companyName AS supplierName,
       curRM.productSKU AS rawMaterialSKU,
       apoc.coll.disjunction(["Product"], labels(myProd))[0] AS usedBy,
       demandQty,
       REDUCE(rmQty = demandQty, rel IN relationships(p) |
         TOINTEGER(ROUND(rmQty / (COALESCE(rel.conversionRatio, 1.0)), 0))
       ) AS rawMaterialQty
ORDER BY usedBy, supplierName, rawMaterialSKU;